# Configuración de paths. 
Busco y agrego el 'root' del Proyecto en sys.path, para evitar problemas al importar mis scripts.

In [2]:
from pathlib import Path
import sys

# Subimos hasta encontrar config.py
base = Path().resolve()
while not (base / "config.py").exists() and base != base.parent:
    base = base.parent

if not (base / "config.py").exists():
    print(f"❌ No se encontró config.py en la jerarquía de carpetas desde: {Path().resolve()}")
    print("➜ sys.path no fue modificado.")
else:
    if str(base) not in sys.path:
        sys.path.insert(0, str(base))
        print(f"✔️ sys.path configurado con raíz del proyecto: {base}")
    else:
        print(f"✔️ sys.path ya está configurado con raíz del proyecto: {base}")

✔️ sys.path ya está configurado con raíz del proyecto: D:\CHardyE-Projects\Python\DataAnalisis\DataAnalisis2025\NYC_Taxi_Lab


# 1. ETL

## 1.1. Extracción.

In [3]:
import pandas as pd
from pathlib import Path

# Ruta al archivo Parquet
ruta_archivo = Path("../data/raw/yellow_tripdata_2025-01.parquet")

try:
    if not ruta_archivo.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo.resolve()}")
    # Carga con motor explícito
    df = pd.read_parquet(ruta_archivo, engine="pyarrow")

    print(" Extracción correcta")
    print(f" Registros cargados: {len(df)}")
    print(f" Columnas: {len(df.columns)}")

except FileNotFoundError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"⚠️ Error inesperado: {e}")

 Extracción correcta
 Registros cargados: 3475226
 Columnas: 20


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [ ]:
df.head(3)

In [5]:
ruta_archivo = Path("../data/raw/taxi_zone_lookup.csv")

try:
    if not ruta_archivo.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo.resolve()}")
    
    df_zonas = pd.read_csv(ruta_archivo)

    print(" Extracción correcta")
    print(f" Registros cargados: {len(df_zonas)}")
    print(f" Columnas: {len(df_zonas.columns)}")

except FileNotFoundError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"⚠️ Error inesperado: {e}")

 Extracción correcta
 Registros cargados: 265
 Columnas: 4


In [6]:
df_zonas.info()
#df_zonas.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


## 1.2. Transformación.

### 1.2.1 Uno mi df_zonas a mi df principal.

In [ ]:
# Agrego las columnas de mi df_zonas a mi df principal.

# Renombrar columnas para claridad
df_zonas.columns = ["LocationID", "Borough", "Zone", "ServiceZone"]

# Mapear zonas de origen y destino
df = df.merge(df_zonas, how="left", left_on="PULocationID", right_on="LocationID")
df = df.rename(columns={"Borough": "PU_Borough", "Zone": "PU_Zone"})

df = df.merge(df_zonas, how="left", left_on="DOLocationID", right_on="LocationID")
df = df.rename(columns={"Borough": "DO_Borough", "Zone": "DO_Zone"})

# Resumen visual por zona de origen
# resumen = df["PU_Zone"].value_counts().head(10)
# print("🗺️ Top 10 zonas de origen:")
# print(resumen)

df.head(3)

### 1.2.2. Reemplazo los códigos numéricos por sus descripciones legibles.
 
Objetivos:
1. Mejorar la legibilidad inmediata mapeando códigos a descripciones.
2. Facilitar el análisis exploratorio
3. Evitar errores de interpretación, eliminando la necesidad de recordar qué significa cada número.

In [7]:
ratecode_map = {
    1: "Standard",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated",
    6: "Group",
    99: "Unknown"
}

df["RateCode"] = df["RatecodeID"].map(ratecode_map).fillna("Unknown")
# Reordenar columnas: insertar RateCode justo después de RatecodeID
cols = df.columns.tolist()
if "RateCode" in cols:
    cols.remove("RateCode")  # Eliminar la versión que quedó al final
idx = cols.index("RatecodeID")
cols = cols[:idx+1] + ["RateCode"] + cols[idx+1:]
df = df[cols]


df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,RateCode,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.6,1.0,Standard,N,229,237,...,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.5,1.0,Standard,N,236,237,...,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.6,1.0,Standard,N,141,141,...,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0


Repitiendo la misma lógica la transformaciona a mis columnas "payment_type", "store_and_fwd_flag"

In [8]:
payment_map = {
        1: "Credit card",
        2: "Cash",
        3: "No charge",
        4: "Dispute",
        5: "Unknown",
        6: "Voided trip"
    }

payment_map_es = {
    1: "T.Crédito",
    2: "Efectivo",
    3: "Sin cargo",
    4: "Reclamación",
    5: "Desconocido",
    6: "Viaje anulado"
}

df["PaymentType"] = df["payment_type"].map(payment_map_es).fillna("Unknown")
# Reordenar columnas: insertar RateCode justo después de RatecodeID
cols = df.columns.tolist()
if "PaymentType" in cols:
    cols.remove("PaymentType")  # Eliminar la versión que quedó al final
idx = cols.index("payment_type")
cols = cols[:idx+1] + ["PaymentType"] + cols[idx+1:]
df = df[cols]

df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,RateCode,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.6,1.0,Standard,N,229,237,...,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.5,1.0,Standard,N,236,237,...,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.6,1.0,Standard,N,141,141,...,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0


In [9]:
# La columna StoreAndFwd (store_and_fwd_flag) indica si los datos del viaje fueron almacenados 
# temporalmente en el dispositivo del taxi antes de ser enviados al servidor central. Esto 
# suele ocurrir por problemas de conectividad en tiempo real.

flag_map = {
        "Y": "Stored and forwarded",
        "N": "Not stored",
        None: "Unknown"
    }
df["StoreAndFwd"] = df["store_and_fwd_flag"].map(flag_map).fillna("Unknown")
# Reordenar columnas: insertar RateCode justo después de RatecodeID
cols = df.columns.tolist()
if "StoreAndFwd" in cols:
    cols.remove("StoreAndFwd")  # Eliminar la versión que quedó al final
idx = cols.index("store_and_fwd_flag")
cols = cols[:idx+1] + ["StoreAndFwd"] + cols[idx+1:]
df = df[cols]

df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,RateCode,store_and_fwd_flag,StoreAndFwd,PULocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.6,1.0,Standard,N,Not stored,229,...,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.5,1.0,Standard,N,Not stored,236,...,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.6,1.0,Standard,N,Not stored,141,...,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0


In [10]:
# Elimino columnas innecesarias
df = df.drop(columns=["RatecodeID"])
df = df.drop(columns=["payment_type"])
df = df.drop(columns=["store_and_fwd_flag"])

df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RateCode,StoreAndFwd,PULocationID,DOLocationID,PaymentType,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.6,Standard,Not stored,229,237,T.Crédito,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.5,Standard,Not stored,236,237,T.Crédito,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.6,Standard,Not stored,141,141,T.Crédito,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0


### 1.2.3. Analisis de Nulos

In [ ]:
df.info()
df.head(3)

In [ ]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

Vamos a corregir ``passenger_count`` partiendo del supuesto que si se registro un cobro el taxi no pudo ir vacio, tuvo que llevar al menos 1 pasajero.
Creo una nueva columna ``pasajero_imputado = True`` que me permitiria filtrar, visualizar o excluir esos registros en análisis sensibles.

In [ ]:
import numpy as np
passenger_null =  df["passenger_count"].isnull().sum()

# Crear columna de trazabilidad
df["pasajero_imputado"] = False
# Detectar registros con passenger_count nulo y fare_amount > 0
condicion = df["passenger_count"].isnull() & (df["fare_amount"] > 0)
# Imputar valor 1 solo en esos casos
df.loc[condicion, "passenger_count"] = 1
# Marcar trazabilidad
df.loc[condicion, "pasajero_imputado"] = True

print("Total imputados:", df["pasajero_imputado"].sum(), " de ", passenger_null)
print("Distribución de passenger_count:")
print(df["passenger_count"].value_counts(dropna=False).sort_index())

In [ ]:
# Ver distribución de Tipo De Pago (payment_type) en esos registros con NaN
df_nulos = df[df["passenger_count"].isnull()]

df_nulos["PaymentType"].value_counts(dropna=False).sort_index()

In [ ]:
# medio redundante esto ya que arriba ya verifico si tienen forma de pago registado.
# Ver si hay cobro en esos casos dudosos (3:sin cargo, 4:disputado)
df_nulos[df_nulos["PaymentType"].isin([3, 4])]["fare_amount"].describe()

Dejo por ahora los valores con NaN, se redujeron significativamente, y no tienen registrado formas de 
pagos, por lo que son candidatos a ser excluidos del analisis.

Analizo ahora los que aparecen con 0.0, ya que tampoco deberian estar registrados los viajes con cero pasajeros.

In [ ]:
df_pasajero_cero = df[df["passenger_count"] == 0.0].copy()

In [ ]:
# verifico si hubo cobros y su variedad cuando no se registraron pasajeros
df_pasajero_cero["fare_amount"].describe()

- count    24656.000000 -> Total de viajes registrados con 0 pasajeros.
- mean        15.494218 -> Tarifa promedio cobrada en esos viajes.
- std         14.833706 -> Alta variabilidad en las tarifas.
- min        -63.230000 -> ⚠️ Tarifa negativa: posible error o ajuste.
- 25%          7.900000
- 50%         11.400000
- 75%         17.000000
- max        500.000000 -> ⚠️ Tarifa extremadamente alta para un viaje sin pasajeros. 

In [ ]:
total_con_cobro_valido = df_pasajero_cero[
    (df_pasajero_cero["fare_amount"] > 0) &     # excluyo viajes sin cobro o con tarifa negativa.
    (df_pasajero_cero["fare_amount"] <= 17.0)   # descarto valores atípicos por encima del tercer cuartil.
].shape[0]
# Resumen
print(f"Total de viajes con passenger_count = 0.0 y cobro válido: {total_con_cobro_valido}")

total_con_cobro_no_valido = df_pasajero_cero[df_pasajero_cero["fare_amount"] < 0].shape[0]
print(f"Total de viajes con passenger_count = 0.0 y cobro no válido: {total_con_cobro_no_valido}")

total_con_cobro_mayor_75 = df_pasajero_cero[df_pasajero_cero["fare_amount"] > 17].shape[0]
print(f"Total de viajes con passenger_count = 0.0 y cobro mayor al 75%: {total_con_cobro_mayor_75}")

In [ ]:
condicion_cero_valido = (
    (df["passenger_count"] == 0.0) &
    (df["fare_amount"] > 0) &
    (df["fare_amount"] <= 17.0)
)
# Imputar valor 1 solo en esos casos de cobro validado
df.loc[condicion_cero_valido, "passenger_count"] = 1
# Marcar trazabilidad
df.loc[condicion_cero_valido, "pasajero_imputado"] = True

total_imputados_cero = condicion_cero_valido.sum()
print("Total imputados desde 0.0 con cobro válido:", total_imputados_cero, " de ", df_pasajero_cero.shape[0])
print("Distribución de passenger_count tras imputación:")
print(df["passenger_count"].value_counts(dropna=False).sort_index())

In [ ]:
total_imputados_cero = condicion_cero_valido.sum()
print("Total imputados desde 0.0 con cobro válido:", total_imputados_cero, " de ", df_pasajero_cero.shape[0])
print("Distribución de passenger_count tras imputación:")
print(df["passenger_count"].value_counts(dropna=False).sort_index())

In [ ]:
df_pasajero_cero[df_pasajero_cero["fare_amount"] > 17]["fare_amount"].describe()

In [ ]:
df_pasajero_cero_alto = df_pasajero_cero[df_pasajero_cero["fare_amount"] > 17].copy()
# Total general del subconjunto
print(f"Total de registros con passenger_count = 0.0 y cobro alto: {df_pasajero_cero_alto.shape[0]}")
# Cobros válidos (positivos)
total_valido_alto = df_pasajero_cero_alto[df_pasajero_cero_alto["fare_amount"] < 37.30].shape[0]
print(f"Total de viajes con passenger_count = 0.0 y cobro < 37.30 válido: {total_valido_alto}")

In [ ]:
condicion_cero_alto_valido = (
    (df["passenger_count"] == 0.0) &
    (df["fare_amount"] > 17.0) &
    (df["fare_amount"] < 37.30)
)

df.loc[condicion_cero_alto_valido, "passenger_count"] = 1
# Marcar trazabilidad
df.loc[condicion_cero_alto_valido, "pasajero_imputado"] = True
# Resumen
total_imputados_cero_alto = condicion_cero_alto_valido.sum()
print("Total imputados desde 0.0 con cobro entre 17.0 y 37.30:", total_imputados_cero_alto, " de ", df_pasajero_cero_alto.shape[0])
print("Distribución de passenger_count tras imputación extendida:")
print(df["passenger_count"].value_counts(dropna=False).sort_index())


In [ ]:
df["registro_invalido"] = False

condicion_nan_no_imputado = df["passenger_count"].isnull() & (~df["pasajero_imputado"])
df.loc[condicion_nan_no_imputado, "registro_invalido"] = True

condicion_cero_excesivo = (df["passenger_count"] == 0.0) & (df["fare_amount"] > 37.30)
df.loc[condicion_cero_excesivo, "registro_invalido"] = True

print("Total de registros marcados como inválidos:", df["registro_invalido"].sum())
print("Distribución de passenger_count en registros inválidos:")
print(df[df["registro_invalido"]]["passenger_count"].value_counts(dropna=False).sort_index())


In [ ]:
df_validos = df[~df["registro_invalido"]].copy()

print(f"Total de registros válidos: {df_validos.shape[0]}")
print("Distribución de passenger_count en datos válidos:")
print(df_validos["passenger_count"].value_counts(dropna=False).sort_index())


In [ ]:
# Forzar tipos antes de exportar
df["store_and_fwd_flag"] = df["store_and_fwd_flag"].astype(str)

df["VendorID"] = df["VendorID"].astype("int32")
df["passenger_count"] = df["passenger_count"].fillna(0).astype("int32")
df["RatecodeID"] = df["RatecodeID"].fillna(1).astype("int32")
df["payment_type"] = df["payment_type"].astype("int64")

print(df.dtypes)
df.head(3)

In [ ]:
print(df.isnull().sum()) # Muestra valores nulos

In [ ]:
# Elimino columnas innecesarias
df = df.drop(columns=["LocationID", "ServiceZone"])
df = df.drop(columns=["LocationID", "ServiceZone"])
df = df.drop(columns=["RatecodeID"])

## 1.3. Carga.

In [ ]:
# Guardar dataset enriquecido
df.to_csv("../data/processed/yellow_tripdata_enriquecido.csv", index=False)

In [ ]:
df.head(3)
df.info()


# 2 Celda Temp - Tools Varias

## 2.1 Backup

In [12]:
# Temporalmente creo un copia de mi df para ir probando las distintas conversiones y o uniones, 
# y no tener que estar realizando la extraccion nuevamente
df_backup = df.copy(deep=True)
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RateCode,StoreAndFwd,PULocationID,DOLocationID,PaymentType,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.6,Standard,Not stored,229,237,T.Crédito,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.5,Standard,Not stored,236,237,T.Crédito,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.6,Standard,Not stored,141,141,T.Crédito,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0


## 2.2 Restauración

In [ ]:
# Restauro mi df_bkp
df = df_backup.copy(deep=True)
df.head(3)

## 2.3 Peso de los DataFrames

In [11]:
try:
    peso_mb_df = df.memory_usage(deep=True).sum() / (1024 ** 2)
    print(f"Peso del DataFrame df:         {peso_mb_df:.2f} MB")
    peso_mb_df_zonas = df_zonas.memory_usage(deep=True).sum() / (1024 ** 2)
    print(f"Peso del DataFrame df_zonas:   {peso_mb_df_zonas:.2f} MB")
    peso_mb_df_validos = df_validos.memory_usage(deep=True).sum() / (1024 ** 2)
    print(f"Peso del DataFrame df_validos: {peso_mb_df_validos:.2f} MB")
except:
    pass

Peso del DataFrame df:         1121.27 MB
Peso del DataFrame df_zonas:   0.05 MB
